# ES2 Example

This example demonstrates the complete workflow of the ES2 Self-Hosted Python SDK, showcasing its capabilities for encrypted similarity search and vector database operations.

## Import ES2

At very first time, you should install and import `es2` package to use python APIs. Before installing, make sure you have conda installed on your system. For more details, see `SDK installation` section in `Get Started`.

In [ ]:
# !pip install es2

In [ ]:
import es2

## Vector Search

### 1. Initialize ES2

To use the ES2 service, initialization is required. Initialization includes establishing a connection to the ES2 server and configuring settings necessary for vector search. You can set the path and ID of the key for data encryption, presets for operations, query encryption, database encryption, and index type.

In [ ]:
es2.init(
    host="localhost",
    port=50050,
    key_path="./keys",
    key_id="quickstart_key",
)

### 2. Create Index

To insert data into the database, you first need to create an index. An index is defined by its name and the dimensionality of the vectors it will store. The dimensionality must match the size of the vectors you plan to insert. For example, if your vectors have 512 dimensions, the index should be created with `dim=512`. This step ensures the database is properly configured to handle your data.

In [ ]:
index = es2.create_index("quickstart_index", dim=512)

### 3. Insert Data

To populate the database, you need to insert data into the created index. For this example, we will use randomly generated vectors as test data. Each vector should match the dimensionality specified during index creation. Additionally, metadata can be attached to each vector to provide context or additional information. This step demonstrates how to prepare and insert data into the database for testing purposes.

In [ ]:
import numpy as np

# Define a function to generate random vectors
def generate_random_vector(dim):
    if dim <= 16 or dim > 4096:
        raise ValueError(f"Invalid dimension: {dim}.")
    
    vec = np.random.uniform(-1.0, 1.0, dim)
    norm = np.linalg.norm(vec)

    if norm > 0:
        vec = vec / norm

    return vec

# Prepare Data
num_data = 10

db_vectors = [
    generate_random_vector(512) for _ in range(num_data)
]
db_metadata = [f"data_{i+1}" for i in range(num_data)]

# Insert Data
index.insert(db_vectors, metadata=db_metadata)

### 4. Encrypted Similarity Search

To perform a similarity search, you can use a query vector to find the most similar vectors in the database. The `index` object contains the decryption key, enabling the ES2 server to return encrypted scores. These scores are decrypted by the client to retrieve the top-k results along with their indices. This process ensures secure and efficient similarity search operations, even when working with encrypted data.

In [ ]:
query = db_vectors[1]
top_k = 2

result = index.search(query, top_k=top_k, output_fields=["metadata"])
result

### Clean Up

In [ ]:
es2.drop_index("quickstart_index")

In [ ]:
es2.release_key("quickstart_key")